# Covid-19 Dashboard

_____________________________________________________________________

This dashboard aims to showcase a survey on Covid-19 cases affecting the United Kingdom collected by Public Health England (PHE). The data used in the dashboard has been extrapolated from the official UK Government website for data and insights on Coronavirus (COVID-19).

The purpose of this dashboard is to present an extract of the data surveyed on the case rate pertaining to gender and age, using the latest data available from PHE. The figures represent a sample of the data collected UK-wide and is viewed to be a general representation of the Covid-19 epidemic affecting the population of the United Kingdom. 

The data illustrated in this graph was collected during the period from January 2020 to November 2020.


__________________________________________________________________________________________

# Cases By Gender and Age Distribution - UK 

### Overview

This graph presents a breakdown of Covid-19 cases in the United Kingdom by Gender and Age. 

The Gender category is defined by males and females in this survey. 

The Age distribution range is from nought to 90+ in this survey.

In [1]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
# Import all libraries in the first instance

In [2]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100
# Mandatory for graphs

In [3]:
with open("caseratebygender.json", "rt") as INFILE:
    data=json.load(INFILE)
# Json file needs to be loaded    

In [4]:
# this function wrangles the data into dataframe
def wrangle_data(data):
    datadic=data['data'][0] 
    
    males=datadic['males']
    females=datadic['females']
    
    ageranges=[x['age'] for x in males]
    
    def min_age(agerange):
        agerange=agerange.replace('+', '')
        start=agerange.split('_')[0]
        return int(start)
    
    ageranges.sort(key=min_age)
    caseratebygender_df=pd.DataFrame(index=ageranges, columns=['males', 'females', 'total'])

    for entry in males:
        ageband=entry['age']
        caseratebygender_df.loc[ageband, 'males']=entry['value']
    
    for entry in females:
        ageband=entry['age']
        caseratebygender_df.loc[ageband, 'females']=entry['value']
    
    caseratebygender_df['total']=caseratebygender_df['males']+caseratebygender_df['females']    

    return caseratebygender_df

caseratebygender_df=wrangle_data(data)

In [5]:
# this function plots and uses widget to design graph
gendercols=wdg.SelectMultiple(
    options=['males', 'females', 'total'],
    value=['males', 'females'],
    rows=3,
    description='Gender',
    disabled=False
)

def plot_gender_graph(graphcolumns):
    ncols=len(graphcolumns)
    if ncols>0:
        caseratebygender_df.plot(kind='bar', y=list(graphcolumns))
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
output=wdg.interactive_output(plot_gender_graph, {'graphcolumns': gendercols})

display(gendercols, output)

# this function refreshes data in graph
def refresh_graph():
    current=gendercols.value
    if current==gendercols.options[0]:
        other=gendercols.options[1]
    else:
        other=gendercols.options[0]
    gendercols.value=other
    gendercols.value=current

    
graph=wdg.interactive_output(plot_gender_graph, {'graphcolumns': gendercols})

#display(gendercols, graph)



SelectMultiple(description='Gender', index=(0, 1), options=('males', 'females', 'total'), rows=3, value=('male…

Output()

Please refresh the data to view the latest version of the graph.

In [6]:
# this function accesses api data
def access_api():
    filters = [
    'areaType=nation',
    'areaName=England'
    ]

    structure = {
    "males": "maleCases",
    "females": "femaleCases"
    }
    
    api = Cov19API(filters=filters, structure=structure)
    caseratebygender=api.get_json()
    return caseratebygender

# this widget displays the refresh data button
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='primary',
    tooltip='Click to download current Public Health England data',
    icon='download'
) 

# this is the api data callback function
def api_button_callback(button):
    apidata=access_api()
    global caseratebygender_df
    caseratebygender_df=wrangle_data(data)
    #refresh_graph()
    apibutton.icon="check"
    
apibutton.on_click(api_button_callback)

display(apibutton)

Button(button_style='primary', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Clic…

_______________________________________________________________________

### Results

The results displayed in the graph indicate a dispoportionately higher number of Covid-19 cases reported in females within the age bracket of 20-24yrs. 

Please note the results are indicative of the cases reported and registered with PHE during this specific time period.

__

### Contact 

Thank you to Public Health England for making the data publically available. 

Please contact PHE for any queries concerning the data.

__

**Author and Copyright Notice** (C) Christa Dobson, 2020 (ec20869@qmul.ac.uk). All rights reserved.: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*